```
In this notebook, we will perform Data Validation test with Python's Great Expectation (GX)
```

# 1. Instantiate Data Context

In [1]:
# Create a data context

from great_expectations.data_context import FileDataContext

context = FileDataContext.create(project_root_dir='./')

# 2. Connect to a Datasource

In [2]:
# Give a name to a Datasource. This name must be unique between Datasources.
datasource_name = 'csv-milestone-3'
datasource = context.sources.add_pandas(datasource_name)

# Give a name to a data asset
asset_name = 'bike-sales'
path_to_data = 'D:\project-m3\P2M3_arcana_data_clean.csv'
asset = datasource.add_csv_asset(asset_name, filepath_or_buffer=path_to_data)

# Build batch request
batch_request = asset.build_batch_request()

# 3. Create an Expectation Suite

In [3]:
# Creat an expectation suite
expectation_suite_name = 'expectation-bike-dataset'
context.add_or_update_expectation_suite(expectation_suite_name)

# Create a validator using above expectation suite
validator = context.get_validator(
    batch_request = batch_request,
    expectation_suite_name = expectation_suite_name
)

# Check the validator
validator.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,date,day,month,year,customer_age,age_group,customer_gender,country,state,product_category,sub_category,product,order_quantity,unit_cost,unit_price,profit,cost,revenue
0,2013-11-26,26,November,2013,19,Youth (<25),M,Canada,British Columbia,Accessories,Bike Racks,Hitch Rack - 4-Bike,8,45,120,590,360,950
1,2015-11-26,26,November,2015,19,Youth (<25),M,Canada,British Columbia,Accessories,Bike Racks,Hitch Rack - 4-Bike,8,45,120,590,360,950
2,2014-03-23,23,March,2014,49,Adults (35-64),M,Australia,New South Wales,Accessories,Bike Racks,Hitch Rack - 4-Bike,23,45,120,1366,1035,2401
3,2016-03-23,23,March,2016,49,Adults (35-64),M,Australia,New South Wales,Accessories,Bike Racks,Hitch Rack - 4-Bike,20,45,120,1188,900,2088
4,2014-05-15,15,May,2014,47,Adults (35-64),F,Australia,New South Wales,Accessories,Bike Racks,Hitch Rack - 4-Bike,4,45,120,238,180,418


## 3.1 Expectations
### Expectation 1 --> to be unique
```
Expectation di bawah ini digunakan untuk memastikan bahwa setiap nilai dalam kolom 'date' adalah unik, 
artinya tidak ada tanggal yang berulang dalam dataset.
```

In [15]:
# Expectation 1 : Column `date` must be unique
validator.expect_column_values_to_be_unique('date')

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "success": false,
  "result": {
    "element_count": 9866,
    "unexpected_count": 9860,
    "unexpected_percent": 99.93918508007297,
    "partial_unexpected_list": [
      "2013-11-26",
      "2015-11-26",
      "2014-03-23",
      "2016-03-23",
      "2014-05-15",
      "2016-05-15",
      "2014-05-22",
      "2016-05-22",
      "2014-02-22",
      "2016-02-22",
      "2013-07-30",
      "2015-07-30",
      "2013-07-15",
      "2015-07-15",
      "2013-08-02",
      "2015-08-02",
      "2013-09-02",
      "2015-09-02",
      "2014-01-22",
      "2016-01-22"
    ],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 99.93918508007297,
    "unexpected_percent_nonmissing": 99.93918508007297
  }
}

outputnya ```"success": false``` karena kolom ```date``` tidak dapat dianggap unik karena dalam satu tanggal yang sama, bisa terjadi banyak transaksi di berbagai negara atau lokasi, dengan produk dan pelanggan yang berbeda-beda. Oleh karena itu, kemunculan nilai yang berulang dalam kolom seperti ```date```, ```product```, atau ```state``` merupakan hal yang wajar dan mencerminkan karakteristik data transaksi secara umum. 
Maka untuk memastikan tiap baris benar-benar unik (mewakili 1 transaksi), saya akan buat kolom baru yang berisi gabungan kolom-kolom.

In [10]:
df = validator.active_batch.data.dataframe

# Tambah kolom sementara
df['unique_key'] = (
    df['date'].astype(str) + "_" +
    df['product'] + "_" +
    df['state'] + "_" +
    df['order_quantity'].astype(str) + "_" +
    df['unit_price'].astype(str) + "_" +
    df['customer_gender'] + "_" +
    df['customer_age'].astype(str)
)


# Buat validator sementara dari dataframe baru
temp_validator = context.sources.pandas_default.read_dataframe(df)

# Jalankan expectation di validator baru
temp_validator.expect_column_values_to_be_unique('unique_key')



Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "success": true,
  "result": {
    "element_count": 9866,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  }
}

Dengan menggunakan kombinasi kolom sebagai unique_key, tidak ditemukan satupun duplikat, sehingga expectation expect_column_values_to_be_unique berhasil terpenuhi (success: true). Ini mengindikasikan bahwa setiap baris dalam data mewakili satu transaksi yang benar-benar unik dari sisi waktu, produk, lokasi, jumlah pembelian, harga, serta karakteristik pelanggan

### Expectation 2 --> to be between min_value and max_value
```
Expectation di bawah ini digunakan untuk memverifikasi bahwa setiap nilai dalam kolom 'unit_cost'
berada dalam rentang nilai yang diharapkan, yaitu antara 0 hingga 100.
```

In [5]:
# Expectation 2 : Column `unit_cost` must be less than $ 100
validator.expect_column_values_to_be_between(
    column='unit_cost', min_value=0, max_value=100
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "success": true,
  "result": {
    "element_count": 9866,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  }
}

- Validasi kolom unit_cost berhasil, seluruh nilainya berada dalam rentang 0 hingga 100.
- Dari sudut pandang bisnis, kisaran biaya unit antara 0 hingga 100 dianggap masuk akal karena mencerminkan nilai umum dari biaya bahan baku, biaya produksi, atau biaya logistik pada produk-produk konsumen seperti sepeda dan aksesorinya. Banyak perusahaan menerapkan kebijakan untuk menjaga biaya unit tetap dalam kisaran ini agar dapat menetapkan harga jual yang kompetitif, menjaga margin keuntungan, serta mengendalikan struktur biaya secara keseluruhan.

### Expectation 3 --> to be in set
```
Expectation ini digunakan untuk memverifikasi bahwa seluruh nilai pada kolom 'customer_gender' 
terbatas hanya pada dua kategori valid, yaitu 'M' (Male) dan 'F' (Female).
```

In [11]:
# Expectation 3 : Column `customer_gender` must contain one of the following 2 things (M and F)
validator.expect_column_values_to_be_in_set(
    "customer_gender",
    ['M','F']
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "success": true,
  "result": {
    "element_count": 9866,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  }
}

- Validasi terhadap kolom customer_gender berhasil. 
- Seluruh nilai berada dalam kategori yang diharapkan, yaitu 'M' dan 'F', tanpa adanya nilai yang tidak valid atau kosong

### Expectation 4 --> to be in type list
```
Expectation di bawah ini digunakan untuk memverifikasi bahwa seluruh nilai pada kolom 'customer_age'
memiliki tipe data sesuai yang diharapkan, yaitu 'int64' karena usia pelanggan merupakan bilangan bulat tanpa nilai desimal.
```

In [12]:
# Expectation 4 : Column `customer_age` must in form of int64
validator.expect_column_values_to_be_in_type_list(
    'customer_age', 
    ['int64']
    )

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "success": true,
  "result": {
    "observed_value": "int64"
  }
}

- Validasi pada kolom customer_age berhasil. 
- Seluruh nilai memiliki tipe data int64, yang sesuai untuk merepresentasikan usia sebagai bilangan bulat.

### Expectation 5 --> to match strftime format
```
Expectation ini digunakan untuk memverifikasi bahwa seluruh nilai pada kolom 'date' 
mengikuti format standar ISO yang konsisten, yaitu '%Y-%m-%d'
```

In [13]:
# Expectation 6 : Column 'date' must match strftime format

validator.expect_column_values_to_match_strftime_format('date', '%Y-%m-%d')

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "success": true,
  "result": {
    "element_count": 9866,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  }
}

- Validasi format kolom date berhasil. 
- Seluruh nilai sesuai dengan format tanggal standar %Y-%m-%d, tanpa ada nilai yang salah format atau hilang.

### Expectation 6 --> to not match regex
```
Expectation di bawah ini digunakan untuk memastikan bahwa nilai kolom `product_category` tidak
ada karakter spesial/tidak valid
```

In [16]:
# Expectation 6: The 'product_category' column should not contain any characters other than letters, numbers, or spaces.
validator.expect_column_values_to_not_match_regex(
    column='product_category',
    regex='[^a-zA-Z0-9\s]'
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "success": true,
  "result": {
    "element_count": 9866,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  }
}

- Validasi pada kolom product_category berhasil. 
- Seluruh nilai tidak mengandung karakter selain huruf, angka, atau spasi

### Expectation 7 --> ExpectColumnPairValuesAToBeGreaterThanB
```
Expectation di bawah ini digunakan untuk memastikan nilai dari kolom 'revenue'
harus lebih besar dari nilai kolom 'cost' karena pendapatan(revenue) harus 
lebih besar dari total biaya (cost). Jika tidak, berarti terjadi kerugian
```

In [17]:
validator.expect_column_pair_values_A_to_be_greater_than_B(
    column_A='revenue',
    column_B='cost'
)

Calculating Metrics:   0%|          | 0/7 [00:00<?, ?it/s]

{
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "success": true,
  "result": {
    "element_count": 9866,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  }
}

- Validasi terhadap kolom revenue dan cost berhasil. 
- Seluruh nilai revenue terbukti lebih besar dari cost pada setiap baris, sesuai dengan logika bisnis bahwa pendapatan harus melebihi biaya